# Basic Use Case: Question and Answering from a Document

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]


# Setup LLM to use, mixtral using Groq

In [92]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0, # no hallucination 
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=True
    # other params...
)

### Connecting to SQLite database

In [75]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///data/street_tree_db.sqlite")
print(db.dialect)
table_info = db.get_usable_table_names()
print(db.get_usable_table_names())

sqlite
['street_trees']


### Simple chain getting the sql response

In [94]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

execute_query = QuerySQLDataBaseTool(db=db, verbose=True)
write_query = create_sql_query_chain(llm, db)

def get_query(data):
    return data.get('sql_query')

json_schema = {
    "title": "sql query",
    "description": "",
    "type": "object",
    "properties": {
        "sql_query": {
            "type": "string",
            "description": "sql query statement",
        },
    },
    "required": ["sql_query"],
}

structured_llm = llm.with_structured_output(json_schema)

chain = write_query | structured_llm | get_query | execute_query
chain.invoke({"question": "How many species of trees are in San Francisco?"})

[(148,)]

'[(148,)]'

### Augmentating response with LLM

In [95]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
       If the rows are empty just display "I cannot answer because there is no information about it"
       Do not mention table names or columns or SQL result.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query | structured_llm |  get_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "what are the most common species in san francisco"})

[('Platanus x hispanica :: Sycamore: London Plane', 117), ('Lophostemon confertus :: Brisbane Box', 91), ('Eucalyptus citriodora :: Lemon scented eucalyptus', 81), ('Tristaniopsis laurina :: Swamp Myrtle', 52), ("Arbutus 'Marina' :: Hybrid Strawberry Tree", 39)]

'The most common species in San Francisco are Sycamore: London Plane, Brisbane Box, Lemon scented eucalyptus, Swamp Myrtle, and Hybrid Strawberry Tree.'